In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pybamm
import pandas as pd
from matplotlib import style
from scipy.interpolate import interp1d
import matplotlib.ticker as ticker

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [6]:
from pathlib import Path
import pandas as pd

from pathlib import Path
import pandas as pd

def get_test_data(excel_name, sheet_name):
    try:
        BASE_DIR = Path(__file__).resolve().parent
    except NameError:
        BASE_DIR = Path.cwd()

    excel_path = BASE_DIR / f"{excel_name}.xlsx"

    if not excel_path.exists():
        raise FileNotFoundError(f"Excel file not found: {excel_path}")

    df = pd.read_excel(excel_path, sheet_name=sheet_name)
    return df


In [7]:
import pandas as pd
import numpy as np

# df must contain: "Time [s]" and "Current"
# ------------------------------------------------

# ---- Step classification ----
def classify_current(i, tol=1e-6):
    if i > tol:
        return "Charge"
    elif i < -tol:
        return "Discharge"
    else:
        return "Rest"

df = get_test_data("beginning_of_life", "HPPC 951")
df["Step_Type"] = df["Current [A]"].apply(classify_current)

# ---- Identify step changes ----
df["Step_ID"] = (df["Step_Type"] != df["Step_Type"].shift()).cumsum()

# ---- Compress steps ----
step_df = (
    df.groupby("Step_ID")
    .agg(
        Step_Type=("Step_Type", "first"),
        Start_Time_s=("Time [s]", "first"),
        End_Time_s=("Time [s]", "last"),
        Mean_Current_A=("Current [A]", "max"),
    )
    .reset_index(drop=True)
)

# ---- Step duration ----
step_df["Step_Time_s"] = (
    step_df["End_Time_s"] - step_df["Start_Time_s"]
)

# ---- Reorder columns ----
step_df = step_df[
    [
        "Step_Type",
        "Start_Time_s",
        "End_Time_s",
        "Step_Time_s",
        "Mean_Current_A",
    ]
]

# ---- Display result ----
print(step_df)
output_path = "HPPC_steps.xlsx"
step_df.to_excel(output_path, index=False)

print(f"Exported to {output_path}")


     Step_Type  Start_Time_s  End_Time_s  Step_Time_s  Mean_Current_A
0         Rest         0.000       9.903        9.903          0.0000
1    Discharge        10.025      19.921        9.896         -2.4972
2         Rest        20.040      39.935       19.895          0.0000
3       Charge        49.944      73.635       23.691          0.7512
4         Rest        73.751      93.668       19.917          0.0000
..         ...           ...         ...          ...             ...
438       Rest     85044.496   85054.415        9.919          0.0000
439  Discharge     85321.161   85321.161        0.000         -0.7512
440       Rest     85322.177   85911.206      589.029          0.0000
441  Discharge     85921.221   86179.466      258.245         -0.7487
442       Rest     86189.502   86189.502        0.000          0.0000

[443 rows x 5 columns]
Exported to HPPC_steps.xlsx


In [8]:
import pandas as pd
import numpy as np

rows = []

for _, r in step_df.iterrows():

    step_type = r["Step_Type"]
    start_t = float(r["Start_Time_s"])
    end_t = float(r["End_Time_s"])
    duration = float(r["Step_Time_s"])
    mean_current = r["Mean_Current_A"]

    # Skip invalid / zero steps
    if duration <= 0:
        continue

    # -----------------------
    # REST
    # -----------------------
    if step_type == "Rest":
        rows.append({
            "Step_Type": "Rest",
            "Start_Time_s": round(start_t, 3),
            "End_Time_s": round(end_t, 3),
            "Duration_s": round(duration, 3),
            "Current_A": 0.0
        })

    # -----------------------
    # DISCHARGE
    # -----------------------
    elif step_type == "Discharge":
        rows.append({
            "Step_Type": "Discharge",
            "Start_Time_s": round(start_t, 3),
            "End_Time_s": round(end_t, 3),
            "Duration_s": round(duration, 3),
            "Current_A": round((mean_current), 6)
        })

    # -----------------------
    # CHARGE (CC + CV)
    # -----------------------
    elif step_type == "Charge":

        # ---- CC part ----
        rows.append({
            "Step_Type": "Charge",
            "Start_Time_s": round(start_t, 3),
            "End_Time_s": round(end_t, 3),
            "Duration_s": round(duration, 3),
            "Current_A": round(mean_current, 6)
        })

        # ---- CV part (conditional in PyBaMM) ----
        rows.append({
            "Step_Type": "Hold",
            "Start_Time_s": np.nan,
            "End_Time_s": np.nan,
            "Duration_s": np.nan,
            "Current_A": 0.01   # cutoff current
        })

# -------------------------------------------------
# CREATE EXPORT DATAFRAME
# -------------------------------------------------
export_df = pd.DataFrame(
    rows,
    columns=[
        "Step_Type",
        "Start_Time_s",
        "End_Time_s",
        "Duration_s",
        "Current_A"
    ]
)

# -------------------------------------------------
# EXPORT TO EXCEL
# -------------------------------------------------
output_file = "HPPC_PyBaMM_Experiment_with_Timing.xlsx"
export_df.to_excel(output_file, index=False)

print(f"Exported successfully: {output_file}")


Exported successfully: HPPC_PyBaMM_Experiment_with_Timing.xlsx
